In [239]:
from decimal import Decimal


valor_imovel = Decimal('400000')
aluguel = Decimal('2000')

def tax_annual_to_monthly(tax):
    return Decimal(pow(1 + float(tax), 1/12) - 1)

tax_financiamento_anual = Decimal('0.07')
tax_financiamento_mensal = tax_annual_to_monthly(tax_financiamento_anual)

In [240]:
import pandas as pd





prazo_mes = 120
entrada = Decimal('100000.00')
outros_custos = Decimal('10000.00')
valor_financiado = valor_imovel - entrada
amortizacao_sac = valor_financiado / prazo_mes

df = pd.DataFrame({
    'mes': [0],
    'ano': [0],
    'prestacao': [entrada + outros_custos],
    'aluguel': [Decimal('0.00')],
    'juros': [Decimal('0.00')],
    'amortizacao': [Decimal('0.00')],
    'saldo_devedor': [valor_financiado],
    'salario':[Decimal('0.00')],
    'aportes_financiando': [Decimal('0.00')],
    'aportes_alugando': entrada + outros_custos,
    'rendimento_financiando': Decimal('0.00'),
    'investimento_financiando': Decimal('0.00'),
    'valorizacao_financiando': Decimal('0.00'),
    'total_financiando': valor_imovel - valor_financiado,
    'rendimento_alugando': 0,
    'total_alugando': entrada + outros_custos
})


precision = 2
for i in range(1, 361):

    if i%12 == 1:
        selic_anual = Decimal('0.06')
        selic_mensal = tax_annual_to_monthly(selic_anual)
        igpm = Decimal('0.09')
        valorizacao = Decimal('0.03')

    row = {}

    row['ano'] = i//12
    row['mes'] = i    
    row['amortizacao'] = amortizacao_sac if df.iloc[-1]['saldo_devedor'] > 0 else 0
    row['saldo_devedor'] = df.iloc[-1]['saldo_devedor'] - row['amortizacao']
    row['juros'] = round(df.iloc[-1]['saldo_devedor'] * tax_financiamento_mensal, precision)
    row['prestacao'] = row['amortizacao'] + row['juros']
    row['aluguel'] = round(aluguel * (1 + igpm) ** row['ano'], precision)
    row['salario'] = max(row['prestacao'], row['aluguel'])

    row['aportes_financiando'] = row['salario'] - row['prestacao']
    row['rendimento_financiando'] = round(df.iloc[-1]['investimento_financiando'] * selic_mensal, precision)
    row['investimento_financiando'] = df.iloc[-1]['investimento_financiando'] + row['aportes_financiando']  + row['rendimento_financiando']
    row['valorizacao_financiando'] = round(valor_imovel * (1 + valorizacao) ** row['ano'], precision)
    row['total_financiando'] = row['investimento_financiando'] + row['valorizacao_financiando'] - row['saldo_devedor']    

    row['aportes_alugando'] = row['salario'] - row['aluguel']
    row['rendimento_alugando'] = round(df.iloc[-1]['total_alugando'] * selic_mensal, precision)
    row['total_alugando'] = df.iloc[-1]['total_alugando'] + row['rendimento_alugando'] + row['aportes_alugando']

    df = pd.concat([df, pd.DataFrame.from_records([row])], ignore_index=True)


display(df.iloc[-1])

mes                                360
ano                                 30
prestacao                         0.00
aluguel                       26535.36
juros                             0.00
amortizacao                          0
saldo_devedor                     0.00
salario                       26535.36
aportes_financiando           26535.36
aportes_alugando                  0.00
rendimento_financiando        23450.78
investimento_financiando    4867764.10
valorizacao_financiando      970904.99
total_financiando           5838669.09
rendimento_alugando            5145.46
total_alugando              1062239.91
Name: 360, dtype: object